In [1]:
from sklearn.ensemble import RandomForestRegressor

# Обучение модели случайного леса на тренировочной выборке
rfc_model = RandomForestRegressor(
    #n_estimators=100,            # Количество деревьев в лесу. Большее количество деревьев может улучшить точность, но увеличивает время обучения.
    #max_depth=5,                 # Максимальная глубина каждого дерева. Ограничение глубины снижает вероятность переобучения.
    #min_samples_split=200,        # Минимальное число образцов для разделения узла. Большее значение предотвращает разделение узлов с малым числом выборок.
    #min_samples_leaf=2,          # Минимальное количество выборок, которое должно находиться в каждом листе. Увеличение значения делает модель более устойчивой.
    #max_features='sqrt',         # Максимальное количество признаков, используемых при поиске лучшего разбиения. "sqrt" берёт корень из общего числа признаков.
    #max_leaf_nodes=20,           # Максимальное число листьев в каждом дереве. Ограничивает количество конечных узлов, упрощая структуру дерева.
    #min_impurity_decrease=0.01,  # Минимальное уменьшение нечистоты, требуемое для разделения. Предотвращает создание слишком мелких узлов.
    #bootstrap=False,              # Использовать бутстрэп (выборка с возвращением) для создания деревьев. Это повышает устойчивость модели.
    random_state=COMMON_CONFIG.SEED              # Устанавливает начальное значение для генератора случайных чисел, что обеспечивает воспроизводимость результатов.
)
rfc_model.fit(X_train_selected, y_train)

In [ ]:
X_train_selected

In [ ]:
y_train

In [ ]:
y_pred = pd.DataFrame(rfc_model.predict(X_test_selected), index=X_test_selected.index, columns=['predicted_close'])
y_pred

In [ ]:
all_predictions = y_pred#['predicted_close']
all_targets = y_test#['close']
plot(all_predictions, all_targets, "ML model Predictions vs. Actual")

In [ ]:
cb_model = cb.CatBoostRegressor(
    n_estimators=1000,             # Общее количество деревьев (итераций). Меньшее значение снижает вероятность переобучения.
    random_state=COMMON_CONFIG.SEED,               # Устанавливает начальное значение для генератора случайных чисел, что обеспечивает воспроизводимость результатов.
    #learning_rate=0.1,             # Темп обучения. Более низкое значение помогает улучшить стабильность и уменьшить вероятность переобучения.
    #depth=6,                       # Глубина каждого дерева. Меньшая глубина снижает вероятность переобучения.
    #l2_leaf_reg=3.0,               # Коэффициент L2-регуляризации на веса в листьях. Увеличивает штраф за большие веса и снижает переобучение.
    #bagging_temperature=1.0,       # Параметр, контролирующий интенсивность случайности в выборке для каждого дерева. Чем выше значение, тем больше разнообразие деревьев.
    #rsm=0.8,                       # Доля признаков, используемых при обучении каждого дерева. Значение меньше 1 уменьшает переобучение.
    #subsample=0.8,                 # Доля данных, используемых для каждого дерева. Чем меньше значение, тем сильнее регуляризация и выше разнообразие деревьев.
    #verbose=0                      # Отключает вывод в консоль.
)

# Добавляем валидационную выборку как eval_set и используем early_stopping
cb_model.fit(X_train_selected, y_train, eval_set=(X_val_selected, y_val), early_stopping_rounds=50)

In [ ]:
y_pred = pd.DataFrame(cb_model.predict(X_test_selected), index=X_test_selected.index, columns=['predicted_close'])
y_pred

In [ ]:
y_test.tail(3)

In [ ]:
y_pred.tail(3)

In [ ]:
all_predictions = y_pred#['predicted_close']
all_targets = y_test#['close']
plot(all_predictions, all_targets, "ML model Predictions vs. Actual")

In [ ]:
!pip install optuna -q
import optuna

In [ ]:
from sklearn.metrics import mean_squared_error

# Функция цели для оптимизации гиперпараметров CatBoost с учетом переобучения
def objective(trial):
    # Определяем гиперпараметры для поиска
    # params = {
    #     #'iterations': trial.suggest_int('iterations', 100, 1000, 10000),
    #     #'depth': trial.suggest_int('depth', 3, 10),
    #     'depth': trial.suggest_int('depth', 9, 15),
    #     #'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.02, 0.3),
    #     'learning_rate': trial.suggest_discrete_uniform('learning_rate', 0.001, 0.02, 0.001),
    #     #'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
    #     'l2_leaf_reg': trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5),
    #     'min_child_samples': trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32]),
    #     'grow_policy': 'Depthwise',
    #     'iterations': 1000,
    #     'use_best_model': True,
    #     'eval_metric': 'RMSE',
    #     'od_type': 'iter',
    #     'od_wait': 20,
    #     #'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0, 1),
    #     #'rsm': trial.suggest_uniform('rsm', 0.5, 1.0),
    #     #'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
    #     'random_seed': COMMON_CONFIG.SEED,
    #     'verbose': 0,
    #     #'early_stopping_rounds': 50
    # }
    
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }
    
    # Инициализация модели с заданными параметрами
    cb_model = cb.CatBoostRegressor(**params)

    # Обучение модели с валидационной выборкой
    cb_model.fit(X_train_selected.copy(), y_train.copy(), eval_set=(X_val_selected.copy(), y_val.copy()), early_stopping_rounds=100, verbose=0)
    
    loss = mean_squared_error(y_test, cb_model.predict(X_test_selected.copy()), squared=False)
    return loss
    
    # Предсказания на тренировочной и валидационной выборках
    #y_train_pred = cb_model.predict_proba(X_train_selected)[:, 1]
    #y_val_pred = cb_model.predict_proba(X_val_selected)[:, 1]
    
    # Оценка метрики ROC AUC на тренировочной и валидационной выборках
    #train_auc = roc_auc_score(y_train, y_train_pred)
    #val_auc = roc_auc_score(y_val, y_val_pred)
    
    # Штраф за переобучение (разница между тренировочной и валидационной метриками)
    #overfitting_penalty = abs(train_auc - val_auc)
    
    # Целевая функция с учетом штрафа: при переобучении функция уменьшится
    #score = val_auc - overfitting_penalty
    
    #return score

In [ ]:
# Создаем объект исследования
study = optuna.create_study(direction='minimize')#direction='maximize')
study.optimize(objective, n_trials=10, n_jobs=-1)  

# Получаем лучшие параметры и результат
best_cb_model_params = study.best_params
best_loss = study.best_value

print("Лучшие параметры:", best_cb_model_params)
print("Лучший Loss:", best_loss)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
# Инициализация модели CatBoostClassifier с оптимальными гиперпараметрами
optimal_params = study.best_params
cb_model = cb.CatBoostRegressor(
    **optimal_params,
    random_seed=COMMON_CONFIG.SEED,
    verbose=0
)

# Обучение модели на тренировочной выборке с валидационной проверкой для контроля
cb_model.fit(X_train_selected, y_train, eval_set=(X_val_selected, y_val), early_stopping_rounds=50, verbose=0)

In [ ]:
y_pred = pd.DataFrame(cb_model.predict(X_test_selected), index=X_test_selected.index, columns=['predicted_close'])
y_pred

In [ ]:
all_predictions = y_pred#['predicted_close']
all_targets = y_test#['close']
plot(all_predictions, all_targets, "ML model Predictions vs. Actual")